In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
import platform

if platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
elif platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
else:
    print('Check your OS system')

# covid data crawling

In [2]:
browser = webdriver.Chrome('c:/driver/chromedriver.exe')
results = []

url = "https://www.worldometers.info/coronavirus/#countries" 
browser.get(url)
time.sleep(2)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
table_list = soup.select('#main_table_countries_today > tbody:nth-child(2) > tr')

for i in range(0, len(table_list)):
    if table_list[i].select('a.mt_a'):
        country  = table_list[i].select('a.mt_a')[0].text
        tot_cases  = table_list[i].select('td.sorting_1')[0].text
        new_cases  = table_list[i].select('tr > td:nth-child(4)')[0].text
        tot_deaths = table_list[i].select('tr > td:nth-child(5)')[0].text
        new_deaths = table_list[i].select('tr > td:nth-child(6)')[0].text
        tot_recov = table_list[i].select('tr > td:nth-child(7)')[0].text
        new_recov = table_list[i].select('tr > td:nth-child(8)')[0].text
        tests = table_list[i].select('tr > td:nth-child(13)')[0].text
        pop = table_list[i].select('tr > td:nth-child(15)')[0].text
        data = [country, tot_cases, new_cases, tot_deaths, new_deaths, tot_recov, new_recov, tests, pop]
        results.append(data)
    else:
        continue
        
time.sleep(3)
browser.quit()
df = pd.DataFrame(results)
df.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop"]
df.to_excel('./covid19.xlsx', index = False)

# data 전처리

In [3]:
df = pd.read_excel('./covid19.xlsx')
df.head()
df.tail()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
215,Vanuatu,4,NaN,1,NaN,3,NaN,"23,000","314,491"
216,Marshall Islands,4,NaN,,NaN,4,NaN,NaN,"59,600"
217,Samoa,3,NaN,,NaN,3,NaN,NaN,"199,769"
218,Saint Helena,2,NaN,,NaN,2,NaN,NaN,"6,096"
219,Micronesia,1,NaN,,NaN,1,NaN,NaN,"116,261"


In [4]:
# column명 변경 필요 없도록 xlsx file 변경함
# df.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop"]
# df.head()

In [5]:
# 데이터 속 , 삭제, dtype int로 변경
df['tot_cases'] = df['tot_cases'].str.replace(',', '')
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,"623,029",NaN,"29,274,349",NaN,"513,059,613","332,997,863"
1,India,30907282,"+2,548","409,338",NaN,"30,063,720","+7,590","434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,"534,311",NaN,"17,666,654",NaN,"54,204,686","214,112,172"
3,Russia,5833175,"+24,702","144,492",780.0,"5,236,214","+19,566","156,300,000","145,998,857"
4,France,5813899,NaN,"111,353",NaN,"5,648,649",NaN,"96,853,814","65,422,093"


In [6]:
df['new_cases'] = df['new_cases'].str.replace('+', '')
df['new_cases'] = df['new_cases'].str.replace(',', '')
df.head()
# nan값 생김

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,"623,029",NaN,"29,274,349",NaN,"513,059,613","332,997,863"
1,India,30907282,2548,"409,338",NaN,"30,063,720","+7,590","434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,"534,311",NaN,"17,666,654",NaN,"54,204,686","214,112,172"
3,Russia,5833175,24702,"144,492",780.0,"5,236,214","+19,566","156,300,000","145,998,857"
4,France,5813899,NaN,"111,353",NaN,"5,648,649",NaN,"96,853,814","65,422,093"


In [7]:
df['tot_deaths'] = df['tot_deaths'].str.replace(',', '')
df.head()
# int 변환에 문제생김

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,"29,274,349",NaN,"513,059,613","332,997,863"
1,India,30907282,2548,409338,NaN,"30,063,720","+7,590","434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,534311,NaN,"17,666,654",NaN,"54,204,686","214,112,172"
3,Russia,5833175,24702,144492,780.0,"5,236,214","+19,566","156,300,000","145,998,857"
4,France,5813899,NaN,111353,NaN,"5,648,649",NaN,"96,853,814","65,422,093"


In [8]:
df['new_deaths'] = df['new_deaths'].replace('+', '')
df['new_deaths'] = df['new_deaths'].replace(',', '')
df['new_deaths'] = df['new_deaths'].replace(' ', '')
df.head(30)
# nan 값 있음

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,"29,274,349",NaN,"513,059,613","332,997,863"
1,India,30907282,2548,409338,NaN,"30,063,720","+7,590","434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,534311,NaN,"17,666,654",NaN,"54,204,686","214,112,172"
3,Russia,5833175,24702,144492,780.0,"5,236,214","+19,566","156,300,000","145,998,857"
4,France,5813899,NaN,111353,NaN,"5,648,649",NaN,"96,853,814","65,422,093"
5,Turkey,5486959,NaN,50278,NaN,"5,354,850",NaN,"63,471,889","85,270,920"
6,UK,5155243,NaN,128431,NaN,"4,362,858",NaN,"225,521,761","68,252,627"
7,Argentina,4662937,NaN,99255,NaN,"4,300,823",NaN,"17,699,179","45,621,774"
8,Colombia,4530610,NaN,113335,NaN,"4,276,465",NaN,"21,060,560","51,439,782"
9,Italy,4272163,NaN,127788,NaN,"4,103,949",NaN,"73,582,831","60,370,303"


In [9]:
df['tot_recov'] = df['tot_recov'].str.replace(',', '')
df.head()
# cannot convert float NaN to integer

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,29274349,NaN,"513,059,613","332,997,863"
1,India,30907282,2548,409338,NaN,30063720,"+7,590","434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,"54,204,686","214,112,172"
3,Russia,5833175,24702,144492,780.0,5236214,"+19,566","156,300,000","145,998,857"
4,France,5813899,NaN,111353,NaN,5648649,NaN,"96,853,814","65,422,093"


In [10]:
df['new_recov'] = df['new_recov'].str.replace('+', '')
df['new_recov'] = df['new_recov'].str.replace(',', '')
df.head()
# int 변환 문제 cannot convert float NaN to integer

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,29274349,NaN,"513,059,613","332,997,863"
1,India,30907282,2548,409338,NaN,30063720,7590,"434,058,138","1,393,938,700"
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,"54,204,686","214,112,172"
3,Russia,5833175,24702,144492,780.0,5236214,19566,"156,300,000","145,998,857"
4,France,5813899,NaN,111353,NaN,5648649,NaN,"96,853,814","65,422,093"


In [11]:
df['tests'] = df['tests'].str.replace(',', '')
df.head(60)

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,29274349,NaN,513059613,"332,997,863"
1,India,30907282,2548,409338,NaN,30063720,7590,434058138,"1,393,938,700"
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,54204686,"214,112,172"
3,Russia,5833175,24702,144492,780.0,5236214,19566,156300000,"145,998,857"
4,France,5813899,NaN,111353,NaN,5648649,NaN,96853814,"65,422,093"
5,Turkey,5486959,NaN,50278,NaN,5354850,NaN,63471889,"85,270,920"
6,UK,5155243,NaN,128431,NaN,4362858,NaN,225521761,"68,252,627"
7,Argentina,4662937,NaN,99255,NaN,4300823,NaN,17699179,"45,621,774"
8,Colombia,4530610,NaN,113335,NaN,4276465,NaN,21060560,"51,439,782"
9,Italy,4272163,NaN,127788,NaN,4103949,NaN,73582831,"60,370,303"


In [12]:
df['pop'] = df['pop'].str.replace(',', '')
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,29274349,NaN,513059613,332997863
1,India,30907282,2548,409338,NaN,30063720,7590,434058138,1393938700
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,54204686,214112172
3,Russia,5833175,24702,144492,780.0,5236214,19566,156300000,145998857
4,France,5813899,NaN,111353,NaN,5648649,NaN,96853814,65422093


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     220 non-null    object 
 1   tot_cases   220 non-null    object 
 2   new_cases   59 non-null     object 
 3   tot_deaths  220 non-null    object 
 4   new_deaths  43 non-null     float64
 5   tot_recov   219 non-null    object 
 6   new_recov   54 non-null     object 
 7   tests       210 non-null    object 
 8   pop         220 non-null    object 
dtypes: float64(1), object(8)
memory usage: 15.6+ KB


In [15]:
df.head()

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,NaN,623029,NaN,29274349,NaN,513059613,332997863
1,India,30907282,2548,409338,NaN,30063720,7590,434058138,1393938700
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,54204686,214112172
3,Russia,5833175,24702,144492,780.0,5236214,19566,156300000,145998857
4,France,5813899,NaN,111353,NaN,5648649,NaN,96853814,65422093


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     220 non-null    object 
 1   tot_cases   220 non-null    object 
 2   new_cases   79 non-null     object 
 3   tot_deaths  220 non-null    object 
 4   new_deaths  58 non-null     float64
 5   tot_recov   219 non-null    object 
 6   new_recov   71 non-null     object 
 7   tests       210 non-null    object 
 8   pop         220 non-null    object 
dtypes: float64(1), object(8)
memory usage: 15.6+ KB


In [14]:
# 결측치 확인
df.isnull().sum()
# df.info()

country         0
tot_cases       0
new_cases     161
tot_deaths      0
new_deaths    177
tot_recov       1
new_recov     166
tests          10
pop             0
dtype: int64

In [15]:
# 결측치, " "에 0 대입
df = df.replace(np.nan, 0)
df = df.replace(' ', 0)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     220 non-null    object 
 1   tot_cases   220 non-null    object 
 2   new_cases   220 non-null    object 
 3   tot_deaths  220 non-null    object 
 4   new_deaths  220 non-null    float64
 5   tot_recov   220 non-null    object 
 6   new_recov   220 non-null    object 
 7   tests       220 non-null    object 
 8   pop         220 non-null    object 
dtypes: float64(1), object(8)
memory usage: 15.6+ KB


,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,USA,34766404,0,623029,0.0,29274349,0,513059613,332997863
1,India,30907282,2548,409338,0.0,30063720,7590,434058138,1393938700
2,Brazil,19106971,0,534311,0.0,17666654,0,54204686,214112172
3,Russia,5833175,24702,144492,780.0,5236214,19566,156300000,145998857
4,France,5813899,0,111353,0.0,5648649,0,96853814,65422093
...,...,...,...,...,...,...,...,...,...
215,Vanuatu,4,0,1,0.0,3,0,23000,314491
216,Marshall Islands,4,0,0,0.0,4,0,0,59600
217,Samoa,3,0,0,0.0,3,0,0,199769
218,Saint Helena,2,0,0,0.0,2,0,0,6096


In [17]:
# 값에 dtype int 변경
df[['tot_cases', 'new_cases', 'tot_deaths', 'new_deaths', 'tot_recov', 'new_recov', 'tests', 'pop']] = df[['tot_cases', 'new_cases', 'tot_deaths', 'new_deaths', 'tot_recov', 'new_recov', 'tests', 'pop']].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     220 non-null    object
 1   tot_cases   220 non-null    int64 
 2   new_cases   220 non-null    int64 
 3   tot_deaths  220 non-null    int64 
 4   new_deaths  220 non-null    int64 
 5   tot_recov   220 non-null    int64 
 6   new_recov   220 non-null    int64 
 7   tests       220 non-null    int64 
 8   pop         220 non-null    int64 
dtypes: int64(8), object(1)
memory usage: 15.6+ KB


# gdp 데이터 정리 전처리

In [18]:
import requests

url='https://www.un.org/en/about-us/member-states'
res = requests.get(url)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

country_names = soup.select('.mb-0')
country_list = []
for i in country_names:
    country_list.append(i.text)

col = ['country']
country_df = pd.DataFrame(country_list, columns=col)

# 영국의 경우 UN에 북아일랜드와 함께 가입되어있는데 gdp 자료는 영국 기준이라서 이름 변경
country_df.iloc[182, 0] = 'United Kingdom'

# 국가별 2018, 2019 gdp 정보가 있는 엑셀 파일 read
gdp_cap = pd.read_excel('gdp_per_capita.xls')
gdp_df = pd.merge(left=country_df,
                right=gdp_cap,
                how='left',
                left_on='country',
                right_on='Country Name')

# df_nulls = gdp_df.isnull().sum()
# un 가입국이지만 gdp 정보가 없는 국가 3개
# 2018년, 2019년 모두 gdp 정보가 없는 국가 drop 필요
# thresh: 해당 row에서 NaN이 아닌 값이 최소 3개 이상 나와야 한다는 설정
gdp_pop = gdp_df.dropna(axis = 0, thresh = 3)
del gdp_pop['Country Name']
gdp_pop = gdp_pop.reset_index()
del gdp_pop['index']

# gdp_pop[gdp_pop['2019'].isnull()]
# 2019 nan값을 2018 정보로 대체
gdp_pop.iloc[94, 2] = gdp_pop.iloc[94, 1]
gdp_pop.iloc[183, 2] = gdp_pop.iloc[183, 1]
del gdp_pop['2018']

# 데이터 타입 변환
gdp_pop = gdp_pop.astype({'2019':'int64'})

In [25]:
country_list

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 "Côte D'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 "Democratic People's Republic of Korea",
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia (Republic of The)',
 'Georgia',
 'Germany',
 'Ghana',
 '

# table join

In [19]:
covid_gdp = pd.merge(df, gdp_pop, how='left', on='country')
covid_gdp

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,USA,34766404,NaN,623029,NaN,29274349,NaN,513059613,332997863,NaN
1,India,30907282,2548,409338,NaN,30063720,7590,434058138,1393938700,2100.0
2,Brazil,19106971,NaN,534311,NaN,17666654,NaN,54204686,214112172,8897.0
3,Russia,5833175,24702,144492,780.0,5236214,19566,156300000,145998857,NaN
4,France,5813899,NaN,111353,NaN,5648649,NaN,96853814,65422093,40380.0
...,...,...,...,...,...,...,...,...,...,...
215,Vanuatu,4,NaN,1,NaN,3,NaN,23000,314491,3102.0
216,Marshall Islands,4,NaN,,NaN,4,NaN,NaN,59600,4073.0
217,Samoa,3,NaN,,NaN,3,NaN,NaN,199769,4324.0
218,Saint Helena,2,NaN,,NaN,2,NaN,NaN,6096,NaN


In [108]:
# 기존 df(covid)와 gdp df사이의 country 명 일치하지 않는 60개 data 확인 
covid_gdp.isnull().sum()

country        0
tot_cases      0
new_cases      0
tot_deaths     0
new_deaths     0
tot_recov      0
new_recov      0
tests          0
pop            0
2019          60
dtype: int64

# Dataframe 정제1

In [27]:
covid_gdp.head()

NameError: name 'covid_gdp' is not defined

In [109]:
# 이름 바꿔야하는 국가명 확인
# 60 개
covid_gdp[covid_gdp['2019'].isnull()]

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,USA,34733831,1078,622850,5,29244223,120,511818976,332992570,NaN
4,Russia,5808473,25140,143712,710,5216648,16429,155900000,145998686,NaN
6,UK,5155243,34471,128431,6,4362858,4304,225521761,68251655,NaN
12,Iran,3394279,20829,86041,182,3041541,15630,24594420,85093355,NaN
20,Czechia,1669496,147,30331,0,1637123,2,30859140,10729366,NaN
39,UAE,651762,1542,1870,4,629809,1519,61034142,10012737,NaN
47,Bolivia,454241,646,17219,19,380500,1767,1924644,11835969,NaN
63,Palestine,315160,0,3582,0,309910,0,1928917,5222469,NaN
66,Venezuela,284942,0,3290,0,267628,0,3359014,28353197,NaN
67,Egypt,283102,0,16396,0,218412,0,3068679,104306960,NaN


df.iloc[0][3]

In [26]:
df_1 = df.copy()
df_1.loc[85,"country"]

'Nigeria'

In [19]:
# excel file 에서 40개 국가명 확인 후 변경
df_1 = df.copy()
df_1.loc[0,"country"] = "United States of America"
df_1.loc[4,"country"] = "Russian Federation"
df_1.loc[6,"country"] = "United Kingdom"
df_1.loc[12,"country"] = "Iran (Islamic Republic of)"
df_1.loc[20,"country"] = "Czech Republic" #Czechia
df_1.loc[40,"country"] = "United Arab Emirates"
df_1.loc[47,"country"] = "Bolivia (Plurinational State of)"
#df_1.loc[63,"country"] = "" #Palestine 국가명 못찾음
df_1.loc[66,"country"] = "Venezuela, Bolivarian Republic of"
df_1.loc[67,"country"] = "Egypt, Arab Rep."
df_1.loc[75,"country"] = "Republic of Moldova"
df_1.loc[85,"country"] = "Republic of Korea"
df_1.loc[88,"country"] = "Kyrgyz Republic" #Kyrgyzstan
# df_1.loc[113,"country"] = "" #Ivory Coast
df_1.loc[116,"country"] = "Democratic Republic of the Congo" #DRC
# df_1.loc[123,"country"] = "" #Réunion
df_1.loc[126,"country"] = "Viet Nam"
# df_1.loc[127,"country"] = "" #French Guiana
df_1.loc[128,"country"] = "Syrian Arab Republic"
# df_1.loc[135,"country"] = "" # Mayotte
df_1.loc[137,"country"] = "French Polynesia" # 이건 이름 같은데 왜?
# df_1.loc[139,"country"] = "" #Guadeloupe
# df_1.loc[141,"country"] = "" # Taiwan
df_1.loc[151,"country"] = "Curacao"
# df_1.loc[152,"country"] = "" # Martinique
df_1.loc[153,"country"] = "Hong Kong SAR, China"
df_1.loc[156,"country"] = "Aruba" # 있는데 왜 못찾아?
df_1.loc[157,"country"] = "South Sudan" # 있는데 왜 못찾아?
df_1.loc[163,"country"] = "Central African Republic" # CAR
df_1.loc[166,"country"] = "Gambia (Republic of The)"
df_1.loc[167,"country"] = "Eritrea"
df_1.loc[172,"country"] = "Channel Islands"
df_1.loc[176,"country"] = "Gibraltar"
df_1.loc[179,"country"] = "Guinea Bissau"
# df_1.loc[182,"country"] = "" #Laos
df_1.loc[183,"country"] = "Sint Maarten (Dutch part)"
df_1.loc[185,"country"] = "Bermuda"
df_1.loc[186,"country"] = "Turks and Caicos Islands"
# df_1.loc[188,"country"] = "" # Saint Martin	
df_1.loc[190,"country"] = "Saint Vincent and the Grenadines"
df_1.loc[192,"country"] = "Isle of Man"
df_1.loc[193,"country"] = "Netherlands" # Caribbean Netherlands
# df_1.loc[195,"country"] = "" # St. Barth
# df_1.loc[196,"country"] = "" # Faeroe Islands
df_1.loc[197,"country"] = "British Virgin Islands"
df_1.loc[198,"country"] = "Cayman Islands"
df_1.loc[200,"country"] = "United Republic of Tanzania"
# df_1.loc[201,"country"] = "" # Wallis and Futuna
df_1.loc[202,"country"] = "Brunei Darussalam"
df_1.loc[205,"country"] = "New Caledonia"
# df_1.loc[206,"country"] = "" # Anguilla
# df_1.loc[207,"country"] = "" # Falkland Islands
df_1.loc[208,"country"] = "Macao SAR, China"
df_1.loc[209,"country"] = "Greenland"
# df_1.loc[210,"country"] = "" #Vatican City
# df_1.loc[211,"country"] = "" # Saint Pierre Miquelon
# df_1.loc[212,"country"] = "" # Montserrat
# df_1.loc[214,"country"] = "" # Western Sahara
# df_1.loc[218,"country"] = "" # Saint Helena
df_1.loc[219,"country"] = "Micronesia (Federated States of)"

In [20]:
df_1

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop
0,United States of America,34766404,0,623029,0,29274349,0,513059613,332997863
1,India,30907282,2548,409338,0,30063720,7590,434058138,1393938700
2,Brazil,19106971,0,534311,0,17666654,0,54204686,214112172
3,Russia,5833175,24702,144492,780,5236214,19566,156300000,145998857
4,Russian Federation,5813899,0,111353,0,5648649,0,96853814,65422093
...,...,...,...,...,...,...,...,...,...
215,Vanuatu,4,0,1,0,3,0,23000,314491
216,Marshall Islands,4,0,0,0,4,0,0,59600
217,Samoa,3,0,0,0,3,0,0,199769
218,Saint Helena,2,0,0,0,2,0,0,6096


# Table join2

In [21]:
covid_gdp_1 = pd.merge(df_1, gdp_pop, how='left', on='country')
covid_gdp_1

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,United States of America,34766404,0,623029,0,29274349,0,513059613,332997863,65279.0
1,India,30907282,2548,409338,0,30063720,7590,434058138,1393938700,2100.0
2,Brazil,19106971,0,534311,0,17666654,0,54204686,214112172,8897.0
3,Russia,5833175,24702,144492,780,5236214,19566,156300000,145998857,NaN
4,Russian Federation,5813899,0,111353,0,5648649,0,96853814,65422093,11497.0
...,...,...,...,...,...,...,...,...,...,...
215,Vanuatu,4,0,1,0,3,0,23000,314491,3102.0
216,Marshall Islands,4,0,0,0,4,0,0,59600,4073.0
217,Samoa,3,0,0,0,3,0,0,199769,4324.0
218,Saint Helena,2,0,0,0,2,0,0,6096,NaN


In [22]:
# 정제 df(covid)와 gdp df사이의 country 명 일치하지 않는 41개 data 확인 
covid_gdp_1.isnull().sum()

country        0
tot_cases      0
new_cases      0
tot_deaths     0
new_deaths     0
tot_recov      0
new_recov      0
tests          0
pop            0
2019          48
dtype: int64

In [23]:
# gdp data는 존재하나 covid data가 존재하지않는 8개 data 존재 확인
covid_gdp_2 = pd.merge(df_1, gdp_pop, how='right', on='country')
covid_gdp_2
covid_gdp_2[covid_gdp_2['tests'].isnull()]
covid_gdp_2.isnull().sum()

country        0
tot_cases     15
new_cases     15
tot_deaths    15
new_deaths    15
tot_recov     15
new_recov     15
tests         15
pop           15
2019           0
dtype: int64

# Dataframe 정제2

In [24]:
# nan 값을 갖는 data 확인
covid_gdp_1[covid_gdp_1['2019'].isnull()]

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
3,Russia,5833175,24702,144492,780,5236214,19566,156300000,145998857,NaN
39,UAE,653284,1522,1876,6,631294,1485,61297926,10013064,NaN
63,Palestine,315224,0,3582,0,310027,0,1932252,5222794,NaN
66,"Venezuela, Bolivarian Republic of",285910,0,3302,0,268872,0,3359014,28352978,NaN
67,"Egypt, Arab Rep.",283212,0,16403,0,219291,0,3068679,104312241,NaN
84,S. Korea,170296,1150,2046,2,154752,565,10964299,51314587,NaN
88,Kyrgyz Republic,142270,1292,2123,10,119694,923,1399147,6634562,NaN
114,Ivory Coast,48869,0,319,0,48242,0,735556,27046580,NaN
123,Vietnam,32665,466,125,0,9331,0,8434266,98238234,NaN
124,Réunion,31845,0,244,0,29597,0,103924,901878,NaN


In [116]:
# column 2019 data==nan 일경우 행 삭제
covid_gdp_1 = covid_gdp_1.dropna(axis=0)

In [117]:
# 179개의 data 남음
covid_gdp_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 219
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     176 non-null    object 
 1   tot_cases   176 non-null    int32  
 2   new_cases   176 non-null    int32  
 3   tot_deaths  176 non-null    int32  
 4   new_deaths  176 non-null    int32  
 5   tot_recov   176 non-null    int32  
 6   new_recov   176 non-null    int32  
 7   tests       176 non-null    int32  
 8   pop         176 non-null    int32  
 9   2019        176 non-null    float64
dtypes: float64(1), int32(8), object(1)
memory usage: 9.6+ KB


In [118]:
covid_gdp_1 = covid_gdp_1.astype({'2019':'int64'})

In [120]:
# final check
covid_gdp_1

,country,tot_cases,new_cases,tot_deaths,new_deaths,tot_recov,new_recov,tests,pop,2019
0,United States of America,34733831,1078,622850,5,29244223,120,511818976,332992570,65279
1,India,30901311,27404,409287,495,30052461,45261,432317813,1393901660,2100
2,Brazil,19089940,0,533546,0,17588312,0,54204686,214108050,8897
3,France,5812639,0,111325,0,5646473,0,96391415,65421700,40380
4,Russian Federation,5808473,25140,143712,710,5216648,16429,155900000,145998686,11497
...,...,...,...,...,...,...,...,...,...,...
213,Solomon Islands,20,0,0,0,20,0,4500,704073,2344
215,Vanuatu,4,0,1,0,3,0,23000,314471,3102
216,Marshall Islands,4,0,0,0,4,0,0,59599,4073
217,Samoa,3,0,0,0,3,0,0,199765,4324


In [121]:
covid_gdp_1.columns = ["country", "tot_cases", "new_cases", "tot_deaths", "new_deaths", "tot_recov", "new_recov", "tests", "pop", "2019"]
covid_gdp_1.to_excel('./covid_gdp.xlsx', index = False)